In [1]:
DATA_PATH = '../Data/Raw/raw_data0_data_scientist.csv'
PKL_EXPORT_PATH = "../data/processed/0_DataCleaned_df.pkl"
CSV_EXPORT_PATH = "../data/processed/0_DataCleaned_df.csv"
US_STATES_ABBR = '../Data/Raw/us_states.xlsx'

In [2]:
# Packages to import
import pandas as pd 
import numpy as np
import re
import logging
import pickle

In [3]:
# Read data and Print shape
raw_df = pd.read_csv(DATA_PATH).set_index('Unnamed: 0').reset_index(drop=True)
raw_df.shape

(1000, 12)

In [4]:
# Display random answer
raw_df.sample(1).iloc[0]

Job Title                           Data Scientist, Product Engagement
Salary Estimate                 Employer Provided Salary:$123K - $178K
Job Description      At Realtor.com®, we have among the most compre...
Rating                                                             3.7
Company Name                                  Realtor.com Careers\n3.7
Location                                                    Austin, TX
Size                                            1001 to 5000 Employees
Founded                                                           1993
Type of ownership                                    Company - Private
Industry                                 Computer Hardware Development
Sector                                          Information Technology
Revenue                                       Unknown / Non-Applicable
Name: 660, dtype: object

In [5]:
raw_df.sample(10)

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Size,Founded,Type of ownership,Industry,Sector,Revenue
363,Data Scientist I,$95K - $125K (Glassdoor est.),Our Purpose\nWe work to connect and power an i...,4.3,Mastercard\n4.3,"O Fallon, MO",10000+ Employees,1966,Company - Public,Financial Transaction Processing,Financial Services,Unknown / Non-Applicable
507,Data Scientist,Employer Provided Salary:$78K - $184K,Data science at the National Security Agency (...,3.7,National Security Agency\n3.7,"Fort Meade, MD",Unknown,1962,Government,National Agencies,Government & Public Administration,Unknown / Non-Applicable
831,Data Scientist,$76K - $100K (Glassdoor est.),To learn more about Arkansas Blue Cross and Bl...,4.1,Arkansas Blue Cross and Blue Shield\n4.1,"Little Rock, AR",1001 to 5000 Employees,1948,Company - Private,Insurance Carriers,Insurance,$1 to $5 billion (USD)
871,Aspire Tech Program - Data Scientist I,$67K - $96K (Glassdoor est.),Job Description\nLexisNexis Legal & Profession...,4.0,RELX\n4.0,"Miamisburg, OH",10000+ Employees,1880,Company - Public,Information Technology Support Services,Information Technology,$10+ billion (USD)
645,Data Scientist,Employer Provided Salary:$100K,"The Data Scientist will use analytical, statis...",3.6,"Stifel, Nicolaus & Co., Inc.\n3.6","New York, NY",5001 to 10000 Employees,1890,Company - Public,Investment & Asset Management,Financial Services,$1 to $5 billion (USD)
956,"Senior Data Scientist, Analytics",$136K - $183K (Glassdoor est.),Discord is looking for experienced and passion...,3.5,Discord\n3.5,"San Francisco, CA",501 to 1000 Employees,2015,Company - Private,Internet & Web Services,Information Technology,$25 to $100 million (USD)
808,Senior Data Scientist,Employer Provided Salary:$130K - $164K,Company Description\n\nEtsy is the global mark...,4.6,Etsy\n4.6,"Brooklyn, NY",1001 to 5000 Employees,2005,Company - Public,Other Retail Stores,Retail & Wholesale,$1 to $5 billion (USD)
59,"Data Scientist I, Bioinformatics",$81K - $112K (Glassdoor est.),Job Summary\n\nThe Bioinformatics department h...,3.9,UT Southwestern\n3.9,"Dallas, TX",10000+ Employees,1943,Hospital,Health Care Services & Hospitals,Healthcare,$1 to $5 billion (USD)
715,Data Scientist - Artificial Intelligence,-1,Overview:\nThis is a remote position within th...,4.3,Adstra\n4.3,United States,51 to 200 Employees,1978,Company - Private,Enterprise Software & Network Solutions,Information Technology,$25 to $100 million (USD)
238,Data Scientist,$91K - $131K (Glassdoor est.),This position offers the option of a hybrid wo...,3.5,Great West Casualty Company\n3.5,"South Sioux City, NE",1001 to 5000 Employees,1956,Company - Public,Insurance Carriers,Insurance,$1 to $5 billion (USD)


In [6]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Job Title          1000 non-null   object 
 1   Salary Estimate    1000 non-null   object 
 2   Job Description    1000 non-null   object 
 3   Rating             1000 non-null   float64
 4   Company Name       1000 non-null   object 
 5   Location           1000 non-null   object 
 6   Size               1000 non-null   object 
 7   Founded            1000 non-null   int64  
 8   Type of ownership  1000 non-null   object 
 9   Industry           1000 non-null   object 
 10  Sector             1000 non-null   object 
 11  Revenue            1000 non-null   object 
dtypes: float64(1), int64(1), object(10)
memory usage: 93.9+ KB


# Data Cleaning & preprocessing
__________________________________________________________________________________________________________________________________________________


## 1. Job title Column

### 1. Try to classify the job titles in terms of title and seniority
### 2. the seniority column has alot of missing values

In [7]:
raw_df['Job Title'].value_counts()

Data Scientist                                      364
Senior Data Scientist                               110
Decision Scientist                                   57
Aspire Tech Program - Data Scientist I               32
Data Scientist III (Data Products)                   31
Data Scientist, Product Engagement                   30
DATA SCIENTIST I                                     29
Data Scientist I                                     28
Junior Data Scientist/Data Modeler                   28
Jr Data Scientist                                    27
Data Scientist - Fraud                               25
Data Scientist, Product                              24
Senior Data Scientist (2 openings)                   22
Data Scientist I - Telecommute Opportunity           20
Artificial Intelligence (AI) Trainer                 18
Data Analyst                                         17
Senior Data Scientist, Analytics                     13
Senior Data Scientist (Product Analytics)       

In [8]:
def title_simplifier(title):
    if 'data scientist' in title.lower() :
        return 'data scientist'
    elif 'data analyst' in title.lower() or 'analyst' in title.lower():
        return 'data analyst'
    elif 'decision scientist' in title.lower():
        return 'decision scientist'
    elif 'artificial intelligence' in title.lower():
        return 'machine learning engineer'
    elif 'data engineer' in title.lower():
        return 'data engineer'
    elif 'scientist' in title.lower():
        return 'data scientist'
    else:
        return 'na'
    
def seniorty(title):
    if 'senior' in title.lower() or 'sr' in title.lower() or 'lead' in title.lower() or 'principal' in title.lower():
        return 'senior'
    elif 'jr' in title.lower() or 'jr.' in title.lower() or 'junior' in title.lower():
        return 'junior'
    else:
        return 'na'

In [9]:
raw_df['job_title'] = raw_df['Job Title'].apply(title_simplifier)

In [10]:
raw_df.job_title.value_counts()

data scientist               903
decision scientist            57
data analyst                  21
machine learning engineer     18
data engineer                  1
Name: job_title, dtype: int64

In [11]:
raw_df['seniority'] = raw_df['Job Title'].apply(seniorty)

In [12]:
# alot of Na in the seniority column
raw_df['seniority'].value_counts()

na        760
senior    171
junior     69
Name: seniority, dtype: int64

## 2. Salary Column

### 1. Parse the Salary Data Column 
### 2. Remove rows in which salary equal -1
### 3. Remove any Text in the columns
### 4. Change Hourly Rate salary to Yearly rate
### 5. split salary into min and max salary column
### 6. calculate the avg salary

In [13]:
# How many Null values in the salary column
len(raw_df[raw_df['Salary Estimate' ] == '-1'])

222

In [14]:
# Remove Columns where Salary == -1, We Ended up with 778 rows that have salary
raw_df = raw_df[raw_df['Salary Estimate'] != '-1']
len(raw_df)

778

In [15]:
raw_df['Salary Estimate'].sample(20)

504              $68K - $95K (Glassdoor est.)
339             $95K - $125K (Glassdoor est.)
559     Employer Provided Salary:$70K - $140K
122    Employer Provided Salary:$149K - $175K
67              $89K - $128K (Glassdoor est.)
213    Employer Provided Salary:$133K - $185K
748    Employer Provided Salary:$135K - $196K
460            Employer Provided Salary:$100K
476    Employer Provided Salary:$130K - $164K
88     Employer Provided Salary:$125K - $150K
600    Employer Provided Salary:$123K - $178K
415    Employer Provided Salary:$114K - $157K
147             $72K - $102K (Glassdoor est.)
480              $67K - $96K (Glassdoor est.)
951    Employer Provided Salary:$150K - $175K
706            Employer Provided Salary:$183K
949    Employer Provided Salary:$130K - $164K
853             $77K - $106K (Glassdoor est.)
995    Employer Provided Salary:$150K - $200K
648     Employer Provided Salary:$90K - $242K
Name: Salary Estimate, dtype: object

In [16]:
# Use Regex to Remove Text in the Salary Column
salary = raw_df['Salary Estimate'].str.replace(r'[^0-9-_]', '',regex = True)
salary.sample(10)

371     89-128
580     89-128
426    149-175
725    149-175
535       7563
434    135-196
366    150-200
255     70-140
567      68-95
492        100
Name: Salary Estimate, dtype: object

In [17]:
# The Per hour Columns salary became so big after the dot removed
print(salary.loc[52])
print(salary.loc[16])
print(salary.loc[501])

7000-7500
6500-7500
7563


In [18]:
# split the salary column into a list 
list_salary =salary.str.split('-')

In [19]:
# We need to change the per hour salary to per year
# per year rate = no. * 1.8 / 1000
#Loop through the salary list and check for numbers > 1000 (per hour columns) and divide by 100
for s in list_salary:
    for i in range(len(s)):
        if int(s[i]) > 1000:
            s[i] = (int(s[i]) * 1.8)/ 100

In [20]:
print(list_salary.loc[52])
print(list_salary.loc[16])
print(list_salary.loc[501])

[126.0, 135.0]
[117.0, 135.0]
[136.134]


In [21]:
# Create Two new columns( one for min_salary and one for max_salary)
raw_df['min_salary'] = list_salary.apply(lambda x: int(x[0]))
# Takes x[0] if list has only one value,
raw_df['max_salary'] = list_salary.apply(lambda x: int(x[1]) if len(x)>1 else int(x[0]))

In [22]:
list_salary

0      [123, 178]
2      [135, 196]
3      [102, 148]
4       [95, 125]
6           [100]
          ...    
992    [108, 132]
993    [133, 185]
995    [150, 200]
996    [149, 175]
998         [183]
Name: Salary Estimate, Length: 778, dtype: object

In [23]:
# Evaluate the avg salary
raw_df['avg_salary'] = (raw_df.min_salary + raw_df.max_salary) / 2

## 3. Description Column

### 1. Create a new column call Desc_length, contain no3. of characters in each job description

In [24]:
raw_df['Job Description'][0]

"At Realtor.com®, we have among the most comprehensive and accurate coverage of real estate listings and the most engaged users across all the online real estate portals. Our mission is to make buying, selling, renting, and living in homes easier and more rewarding for everyone.\nBuilding your career? Build it better at Realtor.com®. Join us and help change the world of real estate, one home at a time.\nCome to work for http://Realtor.com ![ A leader in online real estate and backed by industry experience and the News Corp Brand, http://Realtor.com\nhttp://Realtor.com currently has an opportunity for a Data Scientist, Product Analytics, reporting to the Director of Analytics for Growth Product. http://Realtor.com is in the middle of aggressive strategic growth driven off of newly integrated entities, and continuing our record growth and consumer engagement will be critical to http://Realtor.com 's strategic success in the industry. Specifically, this role's core responsibilities will f

In [25]:
raw_df['desc_len'] = raw_df['Job Description'].apply(lambda x: len(x))
raw_df['desc_len']

0      1283
2      1202
3      1206
4      3064
6       783
       ... 
992    1305
993    1540
995    1884
996     874
998     544
Name: desc_len, Length: 778, dtype: int64

## 5. Company Name Column

### 1. Remove the trailing Rating from the company name
### 2. remove the \n from the company name

In [26]:
# you can see that companies that has -1 Rating, there name is fine
# but companies that has a Rating, there name ends with "\n3.2"
raw_df[['Rating','Company Name']].sample(40)

,Rating,Company Name
437,-1.0,Juji
91,4.8,Notion\n4.8
539,4.1,Arkansas Blue Cross and Blue Shield\n4.1
462,4.3,Mastercard\n4.3
8,-1.0,"Cal-Tex Protective Coatings, Inc."
492,3.6,"Stifel, Nicolaus & Co., Inc.\n3.6"
893,4.0,Cedar Inc\n4.0
795,3.6,"Stifel, Nicolaus & Co., Inc.\n3.6"
941,3.1,CVS Health\n3.1
748,3.7,Realtor.com Careers\n3.7


In [27]:
# Remove the Rating and /n from the company name tail
raw_df['company_name'] = raw_df['Company Name'].str.replace(r'\n([0-9]\.[0-9])','',regex = True)

In [28]:
len(raw_df['company_name'].unique())
raw_df['company_name'].value_counts().head(60)

Realtor.com Careers                      75
Cedar Inc                                51
CVS Health                               37
Salesforce                               31
Neuberger Berman                         30
Meta                                     30
Mastercard                               29
Health Alliance Plan                     29
Octaura                                  28
Stifel, Nicolaus & Co., Inc.             28
RELX                                     28
Veracity Software                        27
Notion                                   24
current                                  22
National Security Agency                 21
Dataminr                                 20
eimagine                                 19
Juji                                     18
FAST GROWING TREES LLC                   17
Etsy                                     16
Discord                                  13
Arkansas Blue Cross and Blue Shield      11
Visa                            

## 4. Rating Column

In [29]:
raw_df['Rating'].value_counts()

 3.7    103
 4.0     92
 3.9     91
 4.3     67
 4.1     64
 3.5     52
 4.2     51
 3.8     41
-1.0     38
 3.6     37
 3.1     37
 5.0     29
 4.8     24
 4.6     17
 4.4     11
 3.4     10
 3.3      3
 3.2      3
 4.5      3
 4.9      2
 4.7      1
 2.9      1
 3.0      1
Name: Rating, dtype: int64

In [30]:
raw_df.loc[raw_df['Rating'] == -1, 'company_name'].value_counts()

Juji                                 18
Konnectingtree                        9
DLZP Group                            3
Vandved Search                        2
Fabrx Space                           2
Cal-Tex Protective Coatings, Inc.     1
NewLimit                              1
Cilable                               1
Slesha inc                            1
Name: company_name, dtype: int64

In [31]:
raw_df.loc[raw_df['company_name'] == 'Juji', 'Rating'] = 4.0

In [32]:
raw_df.loc[raw_df['company_name'] == 'Konnectingtree', 'Rating'] = 4.4

In [33]:
raw_df.loc[raw_df['Rating']== -1, 'company_name'].value_counts()

DLZP Group                           3
Vandved Search                       2
Fabrx Space                          2
Cal-Tex Protective Coatings, Inc.    1
NewLimit                             1
Cilable                              1
Slesha inc                           1
Name: company_name, dtype: int64

## 6. Company Location Column

### 1. Remove the Name of the States, Keep only the abbreviation
### 2. Deal with anomalies ['Texas', 'United States', 'New York State']

In [34]:
raw_df['Location'].sample(60)

201     Philadelphia, PA
600           Austin, TX
232         New York, NY
260         New York, NY
218         O Fallon, MO
153         New York, NY
900           Austin, TX
274           Austin, TX
601       Miamisburg, OH
478          Atlanta, GA
101    San Francisco, CA
882           Iselin, NJ
415        United States
250          Detroit, MI
892           Austin, TX
166         New York, NY
94                Remote
777         San Jose, CA
903               Remote
595    San Francisco, CA
35     San Francisco, CA
781               Remote
941                Texas
222       Miamisburg, OH
502      Little Rock, AR
785               Remote
306         New York, NY
167          Atlanta, GA
258    San Francisco, CA
870           Austin, TX
776         Brooklyn, NY
197           Dallas, TX
938        Palo Alto, CA
614         New York, NY
893         New York, NY
45           Fremont, CA
320     Indianapolis, IN
211           Austin, TX
30                Remote
151         San Jose, CA


In [35]:
# Remove the name of the States, keep the abbreviations
raw_df['job_state'] = raw_df['Location'].apply(lambda x: x.split(',')[1] if len(x.split(',')) > 1 else x)
raw_df['job_state'] = raw_df['job_state'].str.strip()

In [36]:
# Turns out we have 4 abnormal values [ united States, Texas, Remote, New York State]
raw_df['job_state'].value_counts()

NY                215
CA                104
TX                 97
Remote             96
OH                 39
MI                 30
MO                 29
Texas              28
NJ                 28
MD                 22
IN                 19
GA                 11
AR                 11
NE                  7
United States       7
IL                  6
NC                  5
MN                  5
PA                  5
VA                  5
CO                  3
New York State      2
KY                  1
FL                  1
WI                  1
CT                  1
Name: job_state, dtype: int64

In [37]:
# Turn out all the 7 united states are in lyra health company , after i did my research; it is located in california(CA)
raw_df[raw_df['job_state'] == 'United States']

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Size,Founded,Type of ownership,Industry,Sector,Revenue,job_title,seniority,min_salary,max_salary,avg_salary,desc_len,company_name,job_state
53,Data Scientist - Product,Employer Provided Salary:$114K - $157K,About Lyra Health\nLyra is transforming mental...,4.3,Lyra Health\n4.3,United States,1001 to 5000 Employees,2015,Company - Private,Health Care Services & Hospitals,Healthcare,$100 to $500 million (USD),data scientist,na,114,157,135.5,985,Lyra Health,United States
83,Data Scientist - Product,Employer Provided Salary:$114K - $157K,About Lyra Health\nLyra is transforming mental...,4.3,Lyra Health\n4.3,United States,1001 to 5000 Employees,2015,Company - Private,Health Care Services & Hospitals,Healthcare,$100 to $500 million (USD),data scientist,na,114,157,135.5,985,Lyra Health,United States
160,Data Scientist - Product,Employer Provided Salary:$114K - $157K,About Lyra Health\nLyra is transforming mental...,4.3,Lyra Health\n4.3,United States,1001 to 5000 Employees,2015,Company - Private,Health Care Services & Hospitals,Healthcare,$100 to $500 million (USD),data scientist,na,114,157,135.5,985,Lyra Health,United States
348,Data Scientist - Product,Employer Provided Salary:$114K - $157K,About Lyra Health\nLyra is transforming mental...,4.3,Lyra Health\n4.3,United States,1001 to 5000 Employees,2015,Company - Private,Health Care Services & Hospitals,Healthcare,$100 to $500 million (USD),data scientist,na,114,157,135.5,985,Lyra Health,United States
387,Data Scientist - Product,Employer Provided Salary:$114K - $157K,About Lyra Health\nLyra is transforming mental...,4.3,Lyra Health\n4.3,United States,1001 to 5000 Employees,2015,Company - Private,Health Care Services & Hospitals,Healthcare,$100 to $500 million (USD),data scientist,na,114,157,135.5,985,Lyra Health,United States
415,Data Scientist - Product,Employer Provided Salary:$114K - $157K,About Lyra Health\nLyra is transforming mental...,4.3,Lyra Health\n4.3,United States,1001 to 5000 Employees,2015,Company - Private,Health Care Services & Hospitals,Healthcare,$100 to $500 million (USD),data scientist,na,114,157,135.5,985,Lyra Health,United States
446,Data Scientist - Product,Employer Provided Salary:$114K - $157K,About Lyra Health\nLyra is transforming mental...,4.3,Lyra Health\n4.3,United States,1001 to 5000 Employees,2015,Company - Private,Health Care Services & Hospitals,Healthcare,$100 to $500 million (USD),data scientist,na,114,157,135.5,985,Lyra Health,United States


In [38]:
raw_df['job_state'].replace({'Texas':'TX','New York State' : 'NY', 'United States' : 'CA'}, inplace =True)

In [39]:
raw_df['job_state'].value_counts()

NY        217
TX        125
CA        111
Remote     96
OH         39
MI         30
MO         29
NJ         28
MD         22
IN         19
AR         11
GA         11
NE          7
IL          6
PA          5
MN          5
VA          5
NC          5
CO          3
KY          1
FL          1
WI          1
CT          1
Name: job_state, dtype: int64

In [40]:
us_states_df = pd.read_excel(US_STATES_ABBR)
us_states_df.head(20)

,state,abbr
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,American Samoa,AS
5,California,CA
6,Colorado,CO
7,Connecticut,CT
8,Delaware,DE
9,District of Columbia,DC


In [41]:
raw_df['job_state'] = raw_df.job_state.apply(lambda x: us_states_df[us_states_df['abbr'] == x].values[0][0] + "-" + x if x in us_states_df['abbr'].value_counts() else 'Remote')

In [42]:
raw_df['job_state']

0           Texas-TX
2           Texas-TX
3           Texas-TX
4        Missouri-MO
6        New York-NY
           ...      
992           Remote
993           Remote
995      New York-NY
996      New York-NY
998    California-CA
Name: job_state, Length: 778, dtype: object

## 7. Company Size Column

In [43]:
raw_df['Size'].value_counts()

10000+ Employees           219
1001 to 5000 Employees     193
1 to 50 Employees          107
51 to 200 Employees        105
5001 to 10000 Employees     41
201 to 500 Employees        39
501 to 1000 Employees       36
Unknown                     31
-1                           7
Name: Size, dtype: int64

In [44]:
# National Security Agency 
raw_df.loc[raw_df['Size'] == 'Unknown', 'company_name'].value_counts()

National Security Agency    21
University of Minnesota      4
Fabrx Space                  2
Goodwill                     2
80 Acres Farms               1
Vetro Tech Inc               1
Name: company_name, dtype: int64

In [45]:
raw_df.loc[raw_df['Size'] == '-1', 'company_name'].value_counts()

DLZP Group        3
Vandved Search    2
NewLimit          1
Slesha inc        1
Name: company_name, dtype: int64

In [46]:
# change national security agency and university of minnesota to 10000+ employess
raw_df.loc[raw_df['company_name'] == 'National Security Agency', 'Size'] = '10000+ Employees'

In [47]:
raw_df.loc[raw_df['company_name'] == 'University of Minnesota', 'Size'] = '10000+ Employees'

In [48]:
# turn the unknowns to -1
raw_df.loc[raw_df['Size'] == 'Unknown', 'Size']= '-1'

In [49]:
raw_df.loc[raw_df['Size'] == '-1', 'company_name'].value_counts()

DLZP Group        3
Vandved Search    2
Fabrx Space       2
Goodwill          2
NewLimit          1
80 Acres Farms    1
Slesha inc        1
Vetro Tech Inc    1
Name: company_name, dtype: int64

In [50]:
raw_df['Size'].value_counts(sort = True)

10000+ Employees           244
1001 to 5000 Employees     193
1 to 50 Employees          107
51 to 200 Employees        105
5001 to 10000 Employees     41
201 to 500 Employees        39
501 to 1000 Employees       36
-1                          13
Name: Size, dtype: int64

## 8. Founded Column

### 1. get the age of the company instead of the year found
### 2. Figure Out where the -1 values coming From 
### 3. Search for the Founded year for the companies that have alot of appearances in the df and replace the values

In [51]:
raw_df['age']= raw_df.Founded.apply(lambda x: x if x < 1 else 2023 - x) 

In [52]:
len(raw_df[raw_df['age'] == -1])

49

In [53]:
raw_df[(raw_df['age'] == -1) & (raw_df['job_state'] == 'Remote')].sample(5)

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Size,Founded,Type of ownership,Industry,...,Revenue,job_title,seniority,min_salary,max_salary,avg_salary,desc_len,company_name,job_state,age
263,AI Data Scientist,Employer Provided Salary:$65.00 - $75.00 Per Hour,Role - AI Data Scientist\nLocation : Remote\nD...,-1.0,Vandved Search,Remote,-1,-1,-1,-1,...,-1,data scientist,na,117,135,126.0,570,Vandved Search,Remote,-1
60,Data Scientist - Remote,Employer Provided Salary:$80K - $110K,Data Scientist\nJob Overview\nWe are looking f...,-1.0,DLZP Group,Remote,-1,-1,-1,-1,...,-1,data scientist,na,80,110,95.0,912,DLZP Group,Remote,-1
89,Data Scientist,Employer Provided Salary:$110K - $113K,Greetings from KonnectingTree!\nWe are looking...,4.4,Konnectingtree,Remote,1 to 50 Employees,-1,Company - Public,-1,...,Unknown / Non-Applicable,data scientist,na,110,113,111.5,516,Konnectingtree,Remote,-1
162,Data Scientist,Employer Provided Salary:$125K - $150K,About Blue Water Thinking\nGuided by our princ...,3.5,Blue Water Thinking\n3.5,Remote,1 to 50 Employees,-1,Contract,-1,...,Unknown / Non-Applicable,data scientist,na,125,150,137.5,712,Blue Water Thinking,Remote,-1
16,AI Data Scientist,Employer Provided Salary:$65.00 - $75.00 Per Hour,Role - AI Data Scientist\nLocation : Remote\nD...,-1.0,Vandved Search,Remote,-1,-1,-1,-1,...,-1,data scientist,na,117,135,126.0,570,Vandved Search,Remote,-1


In [54]:
raw_df[(raw_df['age'] == -1) & ~(raw_df['job_state'] == 'Remote')].sample(5)

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Size,Founded,Type of ownership,Industry,...,Revenue,job_title,seniority,min_salary,max_salary,avg_salary,desc_len,company_name,job_state,age
804,Artificial Intelligence (AI) Trainer,$102K - $161K (Glassdoor est.),We are seeking highly motivated and talented A...,4.0,Juji,"San Jose, CA",1 to 50 Employees,-1,Company - Public,-1,...,Less than $1 million (USD),machine learning engineer,na,102,161,131.5,732,Juji,California-CA,-1
378,Artificial Intelligence (AI) Trainer,$102K - $161K (Glassdoor est.),We are seeking highly motivated and talented A...,4.0,Juji,"San Jose, CA",1 to 50 Employees,-1,Company - Public,-1,...,Less than $1 million (USD),machine learning engineer,na,102,161,131.5,732,Juji,California-CA,-1
28,Data Scientist,Employer Provided Salary:$40.00 - $45.00 Per Hour,We are looking for a Data Scientist The Depart...,-1.0,Cilable,"Richmond, VA",1 to 50 Employees,-1,Company - Public,-1,...,Unknown / Non-Applicable,data scientist,na,72,81,76.5,687,Cilable,Virginia-VA,-1
747,Artificial Intelligence (AI) Trainer,$102K - $161K (Glassdoor est.),We are seeking highly motivated and talented A...,4.0,Juji,"San Jose, CA",1 to 50 Employees,-1,Company - Public,-1,...,Less than $1 million (USD),machine learning engineer,na,102,161,131.5,732,Juji,California-CA,-1
287,Artificial Intelligence (AI) Trainer,$102K - $161K (Glassdoor est.),We are seeking highly motivated and talented A...,4.0,Juji,"San Jose, CA",1 to 50 Employees,-1,Company - Public,-1,...,Less than $1 million (USD),machine learning engineer,na,102,161,131.5,732,Juji,California-CA,-1


In [55]:
# 19 out of 49 (-1) coming from remote jobs, remote jobs has no age
len(raw_df[(raw_df['age'] == -1) & (raw_df['job_state'] == 'Remote')])

19

In [56]:
len(raw_df[(raw_df['age'] == -1) & ~(raw_df['job_state'] == 'Remote')])

30

In [57]:
# 18 out of 49 (-1) coming from comoany called Juji`
len(raw_df[(raw_df['age'] == -1) & (raw_df['company_name'] == 'Juji')])

18

In [58]:
# Turns out Juji Company was founded in 2014, so we can change the age to 9 years
raw_df.loc[raw_df['company_name'] == 'Juji', 'age'] = 9

In [59]:
# there is 9 (-1) coming from Konnecting Tree company, which was founded in 2016, which is 7 years
raw_df.loc[raw_df['company_name'] == 'Konnectingtree', 'age'] = 7

In [60]:
# there is 4 (-1) coming from Cadent Company, which was founded in 2016, which is 7 years
raw_df.loc[raw_df['company_name'] == 'Cadent', 'age'] = 7

In [61]:
# we went from 49 (-1) to 18
len(raw_df[(raw_df['age'] == -1)])

18

In [62]:
raw_df['age'].value_counts(sort=True)

 7      90
 30     75
 8      42
 60     37
 67     36
 24     34
 18     34
 25     32
 19     32
 84     30
 57     29
 1      28
 143    28
 133    28
 13     27
 61     21
 14     20
 9      18
-1      18
 51     14
 65     11
 75     11
 71      9
 38      8
 27      8
 6       7
 172     4
 205     4
 94      4
 80      4
 20      3
 122     3
 12      2
 178     2
 17      2
 29      2
 4       2
 10      2
 31      1
 78      1
 131     1
 322     1
 44      1
 21      1
 163     1
 95      1
 108     1
 120     1
 187     1
 52      1
 34      1
 16      1
 46      1
 33      1
 40      1
Name: age, dtype: int64

In [63]:
raw_df.loc[raw_df['age'] == -1 , 'company_name'].value_counts()

DLZP Group             3
Vandved Search         2
ShyftLabs              2
Fabrx Space            2
Goodwill               2
Blue Water Thinking    2
NewLimit               1
80 Acres Farms         1
Cilable                1
Slesha inc             1
Vetro Tech Inc         1
Name: company_name, dtype: int64

## 9.Type of ownership

In [64]:
raw_df['Type of ownership'].value_counts()

Company - Private                 379
Company - Public                  339
Government                         21
Subsidiary or Business Segment      9
Nonprofit Organization              8
-1                                  7
College / University                7
Hospital                            4
Unknown                             2
Contract                            2
Name: Type of ownership, dtype: int64

In [65]:
raw_df.loc[raw_df['Type of ownership'] == '-1', 'Type of ownership'] = '-1'

In [66]:
raw_df.loc[raw_df['Type of ownership'] == 'Unknown', 'Type of ownership'] = '-1'

In [67]:
raw_df.loc[raw_df['Type of ownership'] == 'Contract', 'Type of ownership'] = '-1'

In [68]:
raw_df['Type of ownership'].value_counts()

Company - Private                 379
Company - Public                  339
Government                         21
-1                                 11
Subsidiary or Business Segment      9
Nonprofit Organization              8
College / University                7
Hospital                            4
Name: Type of ownership, dtype: int64

In [69]:
raw_df[raw_df['Type of ownership'] == 'Subsidiary or Business Segment']

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Size,Founded,Type of ownership,Industry,...,Revenue,job_title,seniority,min_salary,max_salary,avg_salary,desc_len,company_name,job_state,age
38,Data Scientist,$94K - $124K (Glassdoor est.),Overview\nTrissential is a trusted partner for...,4.1,Trissential\n4.1,"Minneapolis, MN",201 to 500 Employees,2003,Subsidiary or Business Segment,Business Consulting,...,$25 to $100 million (USD),data scientist,na,94,124,109.0,629,Trissential,Minnesota-MN,20
70,Data Scientist - Remote,Employer Provided Salary:$120K - $180K,Clearance Level Other Category Data Science Lo...,4.0,General Dynamics Information Technology\n4.0,"Falls Church, VA",10000+ Employees,1996,Subsidiary or Business Segment,Information Technology Support Services,...,$10+ billion (USD),data scientist,na,120,180,150.0,990,General Dynamics Information Technology,Virginia-VA,27
94,Data Scientist (REMOTE),Employer Provided Salary:$120K - $133K,Basic Qualifications :\nRequires a Bachelor’s ...,3.8,"General Dynamics Mission Systems, Inc\n3.8",Remote,10000+ Employees,1952,Subsidiary or Business Segment,Aerospace & Defense,...,$1 to $5 billion (USD),data scientist,na,120,133,126.5,1001,"General Dynamics Mission Systems, Inc",Remote,71
134,Data Scientist (REMOTE),Employer Provided Salary:$120K - $133K,Basic Qualifications :\nRequires a Bachelor’s ...,3.8,"General Dynamics Mission Systems, Inc\n3.8",Remote,10000+ Employees,1952,Subsidiary or Business Segment,Aerospace & Defense,...,$1 to $5 billion (USD),data scientist,na,120,133,126.5,1001,"General Dynamics Mission Systems, Inc",Remote,71
154,Data Scientist (REMOTE),Employer Provided Salary:$120K - $133K,Basic Qualifications :\nRequires a Bachelor’s ...,3.8,"General Dynamics Mission Systems, Inc\n3.8",Remote,10000+ Employees,1952,Subsidiary or Business Segment,Aerospace & Defense,...,$1 to $5 billion (USD),data scientist,na,120,133,126.5,1001,"General Dynamics Mission Systems, Inc",Remote,71
178,Staff Data Scientist,Employer Provided Salary:$112K - $207K,Job Title:\nStaff Data Scientist - Activision ...,3.4,Activision\n3.4,"San Francisco, CA",1001 to 5000 Employees,1979,Subsidiary or Business Segment,Video Game Publishing,...,$500 million to $1 billion (USD),data scientist,na,112,207,159.5,1000,Activision,California-CA,44
200,Data Scientist (REMOTE),Employer Provided Salary:$120K - $133K,Basic Qualifications :\nRequires a Bachelor’s ...,3.8,"General Dynamics Mission Systems, Inc\n3.8",Remote,10000+ Employees,1952,Subsidiary or Business Segment,Aerospace & Defense,...,$1 to $5 billion (USD),data scientist,na,120,133,126.5,1001,"General Dynamics Mission Systems, Inc",Remote,71
297,Data Scientist (REMOTE),Employer Provided Salary:$120K - $133K,Basic Qualifications :\nRequires a Bachelor’s ...,3.8,"General Dynamics Mission Systems, Inc\n3.8",Remote,10000+ Employees,1952,Subsidiary or Business Segment,Aerospace & Defense,...,$1 to $5 billion (USD),data scientist,na,120,133,126.5,1001,"General Dynamics Mission Systems, Inc",Remote,71
328,Data Scientist (REMOTE),Employer Provided Salary:$120K - $133K,Basic Qualifications :\nRequires a Bachelor’s ...,3.8,"General Dynamics Mission Systems, Inc\n3.8",Remote,10000+ Employees,1952,Subsidiary or Business Segment,Aerospace & Defense,...,$1 to $5 billion (USD),data scientist,na,120,133,126.5,1001,"General Dynamics Mission Systems, Inc",Remote,71


In [70]:
 raw_df.loc[raw_df['Type of ownership'] == 'Subsidiary or Business Segment', 'Type of ownership'] = 'Company - Private'

In [71]:
raw_df['Type of ownership'].value_counts()

Company - Private         388
Company - Public          339
Government                 21
-1                         11
Nonprofit Organization      8
College / University        7
Hospital                    4
Name: Type of ownership, dtype: int64

In [72]:
raw_df[raw_df['Type of ownership'] == '-1']

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Size,Founded,Type of ownership,Industry,...,Revenue,job_title,seniority,min_salary,max_salary,avg_salary,desc_len,company_name,job_state,age
16,AI Data Scientist,Employer Provided Salary:$65.00 - $75.00 Per Hour,Role - AI Data Scientist\nLocation : Remote\nD...,-1.0,Vandved Search,Remote,-1,-1,-1,-1,...,-1,data scientist,na,117,135,126.0,570,Vandved Search,Remote,-1
17,"Data Scientist, Bioinformatics",Employer Provided Salary:$125K - $170K,About NewLimit\nNewLimit is a biotechnology co...,-1.0,NewLimit,"South San Francisco, CA",-1,-1,-1,-1,...,-1,data scientist,na,125,170,147.5,800,NewLimit,California-CA,-1
30,Data Scientist - Remote,Employer Provided Salary:$80K - $110K,Data Scientist\nJob Overview\nWe are looking f...,-1.0,DLZP Group,Remote,-1,-1,-1,-1,...,-1,data scientist,na,80,110,95.0,912,DLZP Group,Remote,-1
52,Data Scientist,Employer Provided Salary:$70.00 - $75.00 Per Hour,6 month contract to hire with Walmart Inc.\nJo...,-1.0,Slesha inc,Remote,-1,-1,-1,-1,...,-1,data scientist,na,126,135,130.5,538,Slesha inc,Remote,-1
60,Data Scientist - Remote,Employer Provided Salary:$80K - $110K,Data Scientist\nJob Overview\nWe are looking f...,-1.0,DLZP Group,Remote,-1,-1,-1,-1,...,-1,data scientist,na,80,110,95.0,912,DLZP Group,Remote,-1
87,Data Scientist,$92K - $135K (Glassdoor est.),Must live or plan to relocate to NY\n\nThe Dat...,3.3,Goodwill\n3.3,"Rochester, NY",-1,-1,-1,-1,...,Unknown / Non-Applicable,data scientist,na,92,135,113.5,770,Goodwill,New York-NY,-1
88,Data Scientist,Employer Provided Salary:$125K - $150K,About Blue Water Thinking\nGuided by our princ...,3.5,Blue Water Thinking\n3.5,Remote,1 to 50 Employees,-1,-1,-1,...,Unknown / Non-Applicable,data scientist,na,125,150,137.5,712,Blue Water Thinking,Remote,-1
107,Data Scientist,$92K - $135K (Glassdoor est.),Must live or plan to relocate to NY\n\nThe Dat...,3.3,Goodwill\n3.3,"Rochester, NY",-1,-1,-1,-1,...,Unknown / Non-Applicable,data scientist,na,92,135,113.5,770,Goodwill,New York-NY,-1
138,Data Scientist - Remote,Employer Provided Salary:$80K - $110K,Data Scientist\nJob Overview\nWe are looking f...,-1.0,DLZP Group,Remote,-1,-1,-1,-1,...,-1,data scientist,na,80,110,95.0,912,DLZP Group,Remote,-1
162,Data Scientist,Employer Provided Salary:$125K - $150K,About Blue Water Thinking\nGuided by our princ...,3.5,Blue Water Thinking\n3.5,Remote,1 to 50 Employees,-1,-1,-1,...,Unknown / Non-Applicable,data scientist,na,125,150,137.5,712,Blue Water Thinking,Remote,-1


## 10. Industry Column

In [73]:
raw_df['Industry'].value_counts()

Internet & Web Services                    136
Information Technology Support Services     91
Computer Hardware Development               79
Enterprise Software & Network Solutions     68
Financial Transaction Processing            60
Investment & Asset Management               58
Health Care Services & Hospitals            53
Insurance Carriers                          48
-1                                          43
Banking & Lending                           22
National Agencies                           21
Home Furniture & Housewares Stores          17
Other Retail Stores                         16
Energy & Utilities                           9
Aerospace & Defense                          8
Colleges & Universities                      8
Software Development                         8
Business Consulting                          6
Biotech & Pharmaceuticals                    4
Research & Development                       4
Advertising & Public Relations               4
Drug & Health

In [74]:
raw_df.loc[raw_df['Industry'] == '-1', 'company_name'].value_counts()

Juji                   18
Konnectingtree          9
DLZP Group              3
Vandved Search          2
ShyftLabs               2
Fabrx Space             2
Goodwill                2
Blue Water Thinking     2
NewLimit                1
Cilable                 1
Slesha inc              1
Name: company_name, dtype: int64

In [75]:
raw_df.loc[raw_df['company_name'] == 'Juji', 'Industry'] = 'Information Technology Support Services'

In [76]:
raw_df.loc[raw_df['company_name'] == 'Konnectingtree', 'Industry'] = 'Information Technology Support Services'

In [77]:
raw_df.loc[raw_df['Industry'] == '-1', 'company_name'].value_counts()

DLZP Group             3
Vandved Search         2
ShyftLabs              2
Fabrx Space            2
Goodwill               2
Blue Water Thinking    2
NewLimit               1
Cilable                1
Slesha inc             1
Name: company_name, dtype: int64

In [78]:
len(raw_df.loc[raw_df['Industry'] == '-1', 'company_name'])

16

## 11. Sector Column

In [79]:
raw_df['Sector'].value_counts()

Information Technology                         382
Financial Services                             140
Healthcare                                      53
Insurance                                       50
-1                                              43
Retail & Wholesale                              37
Government & Public Administration              21
Management & Consulting                         10
Energy, Mining & Utilities                       9
Education                                        8
Aerospace & Defense                              8
Media & Communication                            5
Pharmaceutical & Biotechnology                   4
Telecommunications                               2
Hotels & Travel Accommodation                    2
Agriculture                                      1
Manufacturing                                    1
Construction, Repair & Maintenance Services      1
Human Resources & Staffing                       1
Name: Sector, dtype: int64

In [80]:
raw_df.loc[raw_df['Sector'] == '-1', 'company_name'].value_counts()

Juji                   18
Konnectingtree          9
DLZP Group              3
Vandved Search          2
ShyftLabs               2
Fabrx Space             2
Goodwill                2
Blue Water Thinking     2
NewLimit                1
Cilable                 1
Slesha inc              1
Name: company_name, dtype: int64

In [81]:
raw_df.loc[raw_df['company_name'] == 'Juji' , 'Sector'] = 'Information Technology'

In [82]:
raw_df.loc[raw_df['company_name'] == 'Konnectingtree' , 'Sector'] = 'Information Technology'

In [83]:
raw_df.loc[raw_df['Sector'] == '-1', 'company_name'].value_counts()

DLZP Group             3
Vandved Search         2
ShyftLabs              2
Fabrx Space            2
Goodwill               2
Blue Water Thinking    2
NewLimit               1
Cilable                1
Slesha inc             1
Name: company_name, dtype: int64

In [84]:
len(raw_df.loc[raw_df['Sector'] == '-1', 'company_name'].value_counts())

9

## 12. Revenue Column

In [85]:
raw_df['Revenue'].value_counts()

Unknown / Non-Applicable            368
$10+ billion (USD)                  166
$1 to $5 billion (USD)              145
$25 to $100 million (USD)            23
$1 to $5 million (USD)               20
Less than $1 million (USD)           19
$500 million to $1 billion (USD)     11
$100 to $500 million (USD)            9
-1                                    7
$5 to $10 billion (USD)               7
$5 to $25 million (USD)               3
Name: Revenue, dtype: int64

In [86]:
raw_df[raw_df['Revenue'] == '-1']

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Size,Founded,Type of ownership,Industry,...,Revenue,job_title,seniority,min_salary,max_salary,avg_salary,desc_len,company_name,job_state,age
16,AI Data Scientist,Employer Provided Salary:$65.00 - $75.00 Per Hour,Role - AI Data Scientist\nLocation : Remote\nD...,-1.0,Vandved Search,Remote,-1,-1,-1,-1,...,-1,data scientist,na,117,135,126.0,570,Vandved Search,Remote,-1
17,"Data Scientist, Bioinformatics",Employer Provided Salary:$125K - $170K,About NewLimit\nNewLimit is a biotechnology co...,-1.0,NewLimit,"South San Francisco, CA",-1,-1,-1,-1,...,-1,data scientist,na,125,170,147.5,800,NewLimit,California-CA,-1
30,Data Scientist - Remote,Employer Provided Salary:$80K - $110K,Data Scientist\nJob Overview\nWe are looking f...,-1.0,DLZP Group,Remote,-1,-1,-1,-1,...,-1,data scientist,na,80,110,95.0,912,DLZP Group,Remote,-1
52,Data Scientist,Employer Provided Salary:$70.00 - $75.00 Per Hour,6 month contract to hire with Walmart Inc.\nJo...,-1.0,Slesha inc,Remote,-1,-1,-1,-1,...,-1,data scientist,na,126,135,130.5,538,Slesha inc,Remote,-1
60,Data Scientist - Remote,Employer Provided Salary:$80K - $110K,Data Scientist\nJob Overview\nWe are looking f...,-1.0,DLZP Group,Remote,-1,-1,-1,-1,...,-1,data scientist,na,80,110,95.0,912,DLZP Group,Remote,-1
138,Data Scientist - Remote,Employer Provided Salary:$80K - $110K,Data Scientist\nJob Overview\nWe are looking f...,-1.0,DLZP Group,Remote,-1,-1,-1,-1,...,-1,data scientist,na,80,110,95.0,912,DLZP Group,Remote,-1
263,AI Data Scientist,Employer Provided Salary:$65.00 - $75.00 Per Hour,Role - AI Data Scientist\nLocation : Remote\nD...,-1.0,Vandved Search,Remote,-1,-1,-1,-1,...,-1,data scientist,na,117,135,126.0,570,Vandved Search,Remote,-1


In [87]:
raw_df.loc[raw_df['Revenue'] == 'Unknown / Non-Applicable', 'company_name'].value_counts()

Realtor.com Careers                  75
Cedar Inc                            51
Mastercard                           29
Octaura                              28
Veracity Software                    27
Notion                               24
current                              22
National Security Agency             21
Dataminr                             20
FAST GROWING TREES LLC               17
Tencent                              10
Konnectingtree                        9
Hyperspace Ventures                   5
University of Minnesota               4
Cadent                                4
ShyftLabs                             2
Ascend Analytics, LLC                 2
Moveworks.ai                          2
Ramp Financial                        2
Goodwill                              2
Blue Water Thinking                   2
Fabrx Space                           2
Hilton Grand Vacations                1
Faire                                 1
80 Acres Farms                        1


## Data Cleaning and Preprocessing Notes

### 1. Job Title column ✔️
    a. classify the job titles in terms of title and seniority ✔️
    b. the seniority column has alot of missing values , maybe figure out how to deal with this later or drop it 🛠
### 2. SalaryColumn ✔️
    a. Remove rows in which salary equal -1 ✔️
    b. Remove any Text in the columns  ✔️
    c. Change Hourly rate salaries to Yearly rat ✔️
    d. Create Three new columns min, max, avg salary✔️
### 3.Job Description Column ✔️
    a. Create a desc_length column contains the length of characters in the job Description, to see if their a correlation between the job description length and salary ✔️
    b. maybe we try to preprocess the descriptions, try to get new insights 🛠
### 4. Rating Column
    a. Deal with missing Rating values(-1) 🛠
### 5. Company Name Column ✔️ 
    a. Remove the rating in the company name tail ✔️
    b. Remove the '\n' text in the company name tail ✔️
### 6. Company Location Column✔️ 
    a. keep the state abbreviation only✔️  
    b. deal with the anomalies['texas', 'newyork state', 'united states'] ✔️ 
### 7. Company Size Column
    a. Deal with most of missing values in the size Column ✔️ 
    b. maybe change this column to categories 🛠
    c. what will we do with the remaining 13 missing values 🛠
### 8.Founded column
    a. change the Founded year to how old the company is ✔️
    b. deal with the missing values ✔️
    c. how are gonna deal with the remaining 18 missing values 🛠
    c. maybe we need to turn this column into categories of age 🛠
### 9. Type of ownership column
    a. Convert subsidiary or busniess segment to company private✔️
    b. convert all missing values to -1✔️
    c. deal with missing values and turn into categories🛠
### 10. Industry Column
    a. changed Juji and Konnecting Tree values from -1 to it support services✔️
    b. deal with the remaining 16 missing values🛠
### 11. Sector Column
    a. changed Juji and Konnecting Tree values from -1 to it support services✔️
    b. deal with the remaining 9 missing values🛠
### 12.Revenue Column
    a. there 375 missing values in the Revenus column🛠
    b. How r we gonna deal with that🛠

## Export Data to a CSV File and a Pickle File

In [88]:
raw_df.head()

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Size,Founded,Type of ownership,Industry,...,Revenue,job_title,seniority,min_salary,max_salary,avg_salary,desc_len,company_name,job_state,age
0,"Data Scientist, Product Engagement",Employer Provided Salary:$123K - $178K,"At Realtor.com®, we have among the most compre...",3.7,Realtor.com Careers\n3.7,"Austin, TX",1001 to 5000 Employees,1993,Company - Private,Computer Hardware Development,...,Unknown / Non-Applicable,data scientist,na,123,178,150.5,1283,Realtor.com Careers,Texas-TX,30
2,Senior Data Scientist,Employer Provided Salary:$135K - $196K,"At Realtor.com®, we have among the most compre...",3.7,Realtor.com Careers\n3.7,"Austin, TX",1001 to 5000 Employees,1993,Company - Private,Computer Hardware Development,...,Unknown / Non-Applicable,data scientist,senior,135,196,165.5,1202,Realtor.com Careers,Texas-TX,30
3,Data Analyst,Employer Provided Salary:$102K - $148K,"At Realtor.com®, we have among the most compre...",3.7,Realtor.com Careers\n3.7,"Austin, TX",1001 to 5000 Employees,1993,Company - Private,Computer Hardware Development,...,Unknown / Non-Applicable,data analyst,na,102,148,125.0,1206,Realtor.com Careers,Texas-TX,30
4,Data Scientist I,$95K - $125K (Glassdoor est.),Our Purpose\nWe work to connect and power an i...,4.3,Mastercard\n4.3,"O Fallon, MO",10000+ Employees,1966,Company - Public,Financial Transaction Processing,...,Unknown / Non-Applicable,data scientist,na,95,125,110.0,3064,Mastercard,Missouri-MO,57
6,Data Scientist,Employer Provided Salary:$100K,"The Data Scientist will use analytical, statis...",3.6,"Stifel, Nicolaus & Co., Inc.\n3.6","New York, NY",5001 to 10000 Employees,1890,Company - Public,Investment & Asset Management,...,$1 to $5 billion (USD),data scientist,na,100,100,100.0,783,"Stifel, Nicolaus & Co., Inc.",New York-NY,133


In [89]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 778 entries, 0 to 998
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Job Title          778 non-null    object 
 1   Salary Estimate    778 non-null    object 
 2   Job Description    778 non-null    object 
 3   Rating             778 non-null    float64
 4   Company Name       778 non-null    object 
 5   Location           778 non-null    object 
 6   Size               778 non-null    object 
 7   Founded            778 non-null    int64  
 8   Type of ownership  778 non-null    object 
 9   Industry           778 non-null    object 
 10  Sector             778 non-null    object 
 11  Revenue            778 non-null    object 
 12  job_title          778 non-null    object 
 13  seniority          778 non-null    object 
 14  min_salary         778 non-null    int64  
 15  max_salary         778 non-null    int64  
 16  avg_salary         778 non

In [90]:
raw_df.to_csv(CSV_EXPORT_PATH, index = False)

In [91]:
raw_df.to_pickle(PKL_EXPORT_PATH)